In [7]:
import numpy as np
import pandas as pd
import os
import cv2

In [8]:
import kagglehub


# Download latest version
path = kagglehub.dataset_download("ashutoshpaliwal26/glucoam-unet-mask-disc-croped")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/glucoam-unet-mask-disc-croped


In [9]:
os.listdir(path)

['cropped']

In [10]:
df = pd.read_csv(f"{path}/cropped/with_disc.csv")
df.head()

,Unnamed: 0,names,types,original_name,fundus,fundus_od_seg,fundus_oc_seg
0,4546,ORIGA-1,0,001.jpg,/full-fundus/ORIGA-1.png,/optic-disc/ORIGA-1.png,/optic-cup/ORIGA-1.png
1,4547,ORIGA-2,0,002.jpg,/full-fundus/ORIGA-2.png,/optic-disc/ORIGA-2.png,/optic-cup/ORIGA-2.png
2,4548,ORIGA-3,0,003.jpg,/full-fundus/ORIGA-3.png,/optic-disc/ORIGA-3.png,/optic-cup/ORIGA-3.png
3,4549,ORIGA-4,0,004.jpg,/full-fundus/ORIGA-4.png,/optic-disc/ORIGA-4.png,/optic-cup/ORIGA-4.png
4,4550,ORIGA-5,0,005.jpg,/full-fundus/ORIGA-5.png,/optic-disc/ORIGA-5.png,/optic-cup/ORIGA-5.png


In [11]:
DIR_PATH = f"{path}/cropped"
IMG_SIZE = (256, 256)

images = []
labels = []

print(os.listdir(DIR_PATH))

for img in df["names"]:
    n_image = cv2.imread(f"{DIR_PATH}/normal/{img}.png")
    m_image = cv2.imread(f"{DIR_PATH}/mask/{img}.png")

    normal_gray = cv2.cvtColor(n_image, cv2.COLOR_BGR2GRAY)
    mask_gray = cv2.cvtColor(m_image, cv2.COLOR_BGR2GRAY)

    resize_n_image = cv2.resize(normal_gray, IMG_SIZE)
    resize_m_image = cv2.resize(mask_gray, IMG_SIZE)
    images.append(resize_n_image)
    labels.append(resize_m_image)

['with_disc.csv', 'normal', 'mask']


In [12]:
normal_images = np.array(images)
mask_images = np.array(labels)

normal_images = np.expand_dims(normal_images, -1)
mask_images = np.expand_dims(mask_images, -1)

print("Shape of the Images : ", normal_images.shape, mask_images.shape)

Shape of the Images :  (3102, 256, 256, 1) (3102, 256, 256, 1)


In [13]:
normal_images = normal_images.astype("float32")/255.0
mask_images = mask_images.astype("float32")/255.0

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(normal_images, mask_images, test_size=0.1)

In [15]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

2025-11-16 08:50:39.723791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763283039.928046      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763283039.987345      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [16]:
input_layer = Input(shape=(IMG_SIZE[0], IMG_SIZE[0], 1))

# Encoder
conv1 = Conv2D(64, (3, 3), activation="relu", padding="same")(input_layer)
conv1 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv1)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(128, (3, 3), activation="relu", padding="same")(pool1)
conv2 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv2)
pool2 = MaxPooling2D((2,2))(conv2)

conv3 = Conv2D(256, (3, 3), activation="relu", padding="same")(pool2)
conv3 = Conv2D(256, (3,3), activation="relu", padding="same")(conv3)
pool3 = MaxPooling2D((2,2))(conv3)

conv4 = Conv2D(512, (3, 3), activation="relu", padding="same")(pool3)
conv4 = Conv2D(512, (3,3), activation="relu", padding="same")(conv4)
pool4 = MaxPooling2D((2, 2))(conv4)

# Bottleneck
bottleneck = Conv2D(1024, (3,3), activation="relu", padding="same")(pool4)
bottleneck = Conv2D(1024, (3,3), activation="relu", padding="same")(bottleneck)

# Decoder
upConv1 = Conv2DTranspose(512, (2,2), strides=2, padding="same")(bottleneck)
concat1 = concatenate([upConv1, conv4])
conv5 = Conv2D(512, (3,3), activation="relu", padding="same")(concat1)
conv5 = Conv2D(512, (3, 3), activation="relu", padding="same")(conv5)

upConv2 = Conv2DTranspose(256, (2, 2), strides=2, padding="same")(conv5)
concat2 = concatenate([upConv2, conv3])
conv6 = Conv2D(256, (3,3), activation="relu", padding="same")(concat2)
conv6 = Conv2D(256, (3, 3), activation="relu", padding="same")(conv6)

upConv3 = Conv2DTranspose(128, (2, 2), strides=2, padding="same")(conv6)
concat3 = concatenate([upConv3, conv2])
conv7 = Conv2D(128, (3,3), activation="relu", padding="same")(concat3)
conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv7)

upConv4 = Conv2DTranspose(64, (2, 2), strides=2, padding="same")(conv7)
concat4 = concatenate([upConv4, conv1])
conv8 = Conv2D(64, (3,3), activation="relu", padding="same")(concat4)
conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv8)

# Output
output_layer = Conv2D(1, (1, 1), activation="sigmoid", padding="same")(conv8)

model = Model(inputs=input_layer, outputs=output_layer)

I0000 00:00:1763283054.920188      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [17]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 16, 16,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 16, 16,    │  4,719,616 │ max_pooling2d_3[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 16, 16,    │  9,438,208 │ conv2d_8[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 32, 32,    │  2,097,664 │ conv2d_9[0][0]    │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ conv2d_transpose

 Total params: 31,030,593 (118.37 MB)

 Trainable params: 31,030,593 (118.37 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [19]:
hist = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20


I0000 00:00:1763283073.485341     112 service.cc:148] XLA service 0x7ce814002710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763283073.486161     112 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1763283074.480447     112 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1763283135.444626     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


87/88 ━━━━━━━━━━━━━━━━━━━━ 0s 666ms/step - accuracy: 0.8175 - loss: 0.5224

E0000 00:00:1763283196.545520     111 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763283196.780520     111 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763283197.398996     111 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763283197.657366     111 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 863ms/step - accuracy: 0.8176 - loss: 0.5216

E0000 00:00:1763283239.740667     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763283239.979934     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763283241.076891     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763283241.340585     109 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


88/88 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.8177 - loss: 0.5209 - val_accuracy: 0.8294 - val_loss: 0.4739
Epoch 2/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 61s 687ms/step - accuracy: 0.8170 - loss: 0.5978 - val_accuracy: 0.8294 - val_loss: 5.3142
Epoch 3/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 60s 687ms/step - accuracy: 0.8281 - loss: 0.7057 - val_accuracy: 0.8556 - val_loss: 0.3722
Epoch 4/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 60s 687ms/step - accuracy: 0.8449 - loss: 0.3839 - val_accuracy: 0.8294 - val_loss: 0.4310
Epoch 5/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 60s 687ms/step - accuracy: 0.8278 - loss: 0.4070 - val_accuracy: 0.8294 - val_loss: 0.3768
Epoch 6/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 60s 687ms/step - accuracy: 0.8266 - loss: 0.3791 - val_accuracy: 0.8495 - val_loss: 0.3682
Epoch 7/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 61s 688ms/step - accuracy: 0.8497 - loss: 0.3719 - val_accuracy: 0.8515 - val_loss: 0.3627
Epoch 8/20
88/88 ━━━━━━━━━━━━━━━━━━━━ 61s 688ms/step - accuracy: 0.8533 - loss: 0.3592 - val_accuracy: 0.8565 

In [20]:
model.save("/kaggle/working/unet_model256.keras")

In [21]:
model.save_weights("/kaggle/working/unet256.weights.h5")

In [ ]:
import matplotlib.pyplt as plt

In [6]:
plt.plot(hist)

NameError: name 'plt' is not defined